## 0. DRIVE mount

In [35]:
# 구글 드라이브와 연동
from google.colab import drive
drive.mount("/content/gdrive")

# 연동 잘 되었는지 확인
!ls "gdrive/My Drive/Colab Notebooks/DACON/"


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
k_fashion  LG_predict_complain


In [ ]:
# zip 파일을 먼저 구글 클라우드에 업로드
# 압축파일을 세션 내 드라이브에 압축해제
!unzip "gdrive/My Drive/Colab Notebooks/DACON/LG_predict_complain/data.zip" -d 'data/'

Archive:  gdrive/My Drive/Colab Notebooks/DACON/LG_predict_complain/data.zip
  inflating: data/sample_submission.csv  
  inflating: data/test_err_data.csv  
  inflating: data/test_quality_data.csv  
  inflating: data/train_err_data.csv  
  inflating: data/train_problem_data.csv  
  inflating: data/train_quality_data.csv  


In [ ]:
!pip install pycaret

     |████████████████████████████████| 256kB 5.0MB/s 
     |████████████████████████████████| 276kB 8.5MB/s 
     |████████████████████████████████| 14.2MB 310kB/s 
     |████████████████████████████████| 245kB 50.7MB/s 
     |████████████████████████████████| 1.8MB 48.2MB/s 
     |████████████████████████████████| 174kB 32.8MB/s 
     |████████████████████████████████| 65.8MB 83kB/s 
     |████████████████████████████████| 157.5MB 85kB/s 
     |████████████████████████████████| 102kB 11.8MB/s 
     |████████████████████████████████| 1.6MB 45.1MB/s 
     |████████████████████████████████| 6.8MB 49.3MB/s 
     |████████████████████████████████| 153kB 52.3MB/s 
     |████████████████████████████████| 61kB 8.6MB/s 
     |████████████████████████████████| 348kB 46.4MB/s 
     |████████████████████████████████| 1.1MB 46.9MB/s 
     |████████████████████████████████| 81kB 10.6MB/s 
     |████████████████████████████████| 163kB 54.3MB/s 
     |████████████████████████████████| 604kB 43.6MB/s

In [36]:
from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')


PATH = 'data/'

## 1. Model Train


In [37]:
"""
 학습에 사용되는 모든 컬럼
"""
all_data_col_err = ['err0', 'err1', 'err2', 'err3', 'err4', 'err5', 'err6', 'err7', 'err8', 'err9', 'err10', 
                    'err11', 'err12', 'err13', 'err14', 'err15', 'err16', 'err17', 'err18', 'err19', 'err20', 
                    'err21', 'err22', 'err23', 'err24', 'err25', 'err26', 'err27', 'err28', 'err29', 'err30', 
                    'err31', 'err32', 'err33', 'err34', 'err35', 'err36', 'err37', 'err38', 'err39', 'err40', 'err41']
# all_data_col_errcode = ['A','B','C','D','UNKNOWN']
all_data_col_errcode = ['A','B']
all_data_col_err_weekday = ['err_week_0', 'err_week_1', 'err_week_2', 'err_week_3', 'err_week_4', 'err_week_5', 'err_week_6']
all_data_col_err_hours = ['err_hours_0', 'err_hours_1', 'err_hours_2', 'err_hours_3', 'err_hours_4', 'err_hours_5', 'err_hours_6', 
                          'err_hours_7', 'err_hours_8', 'err_hours_9', 'err_hours_10', 'err_hours_11', 'err_hours_12', 'err_hours_13', 
                          'err_hours_14', 'err_hours_15', 'err_hours_16', 'err_hours_17', 'err_hours_18', 'err_hours_19', 'err_hours_20', 
                          'err_hours_21', 'err_hours_22', 'err_hours_23']

all_data_col_model = ['model_0', 'model_1', 'model_2', 'model_3', 'model_4', 'model_5', 'model_6', 'model_7', 'model_8']
all_data_col_fwver = ['03.11.1141', '03.11.1149', '03.11.1167', '04.16.2641', '04.16.3345', '04.16.3439', 
                      '04.16.3553', '04.16.3569', '04.16.3571', '04.22.1170', '04.22.1442', '04.22.1448', 
                      '04.22.1478', '04.22.1608', '04.22.1656', '04.22.1666', '04.22.1684', '04.22.1750', 
                      '04.22.1772', '04.22.1778', '04.33.1095', '04.33.1125', '04.33.1149', '04.33.1171', 
                      '04.33.1185', '04.33.1261', '04.73.2237', '04.73.2569', '04.73.2571', '04.73.2577', 
                      '04.82.1684', '04.82.1730', '04.82.1778', '05.15.2090', '05.15.2092', '05.15.2114', 
                      '05.15.2120', '05.15.2122', '05.15.2138', '05.15.3104', '05.66.3237', '05.66.3571', 
                      '10', '10.22.1770', '10.22.1780', '8.5.3']


all_data_col_qual = ['quality_0', 'quality_1', 'quality_2', 'quality_3', 'quality_4', 'quality_5', 'quality_6', 
                     'quality_7', 'quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12']
all_data_col_qual_weekday = ['qual_week_0', 'qual_week_1', 'qual_week_2', 'qual_week_3', 'qual_week_4', 'qual_week_5', 'qual_week_6']
all_data_col_qual_hours = ['qual_hours_0', 'qual_hours_1', 'qual_hours_2', 'qual_hours_3', 'qual_hours_4', 'qual_hours_5', 'qual_hours_6', 
                           'qual_hours_7', 'qual_hours_8', 'qual_hours_9', 'qual_hours_10', 'qual_hours_11', 'qual_hours_12', 'qual_hours_13', 
                           'qual_hours_14', 'qual_hours_15', 'qual_hours_16', 'qual_hours_17', 'qual_hours_18', 'qual_hours_19', 'qual_hours_20', 
                           'qual_hours_21', 'qual_hours_22', 'qual_hours_23']


all_data_columns = ['user_id'] + all_data_col_err + all_data_col_errcode + all_data_col_model + all_data_col_fwver + all_data_col_qual

### 1.1 Train data load

In [41]:
# train_err  = pd.read_csv(PATH+'train_err_data.csv')
df  = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/20210203_train_data_merge_2.csv')

problem = df['problem']
df.drop(['user_id','problem'], axis=1,inplace=True)
df.drop(all_data_col_fwver,axis=1,inplace=True)
# df.drop(all_data_col_errcode,axis=1,inplace=True)
# df.drop(all_data_col_qual_weekday,axis=1,inplace=True)
# df.drop(all_data_col_qual_hours,axis=1,inplace=True)
drop_list = ['err28', 'quality_3', 'quality_4', 'quality_minus_3', 'quality_minus_4', 'quality_minus_7', 'quality_minus_8', 'quality_minus_9', 'quality_minus_10', 'quality_minus_12']
df.drop(drop_list, axis=1,inplace=True)
display(df.head())

,err0,err1,err2,err3,err4,err5,err6,err7,err8,err9,err10,err11,err12,err13,err14,err15,err16,err17,err18,err19,err20,err21,err22,err23,err24,err25,err26,err27,err29,err30,err31,err32,err33,err34,err35,err36,err37,err38,err39,err40,err41,model_0,model_1,model_2,model_3,model_4,model_5,model_6,model_7,model_8,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,quality_minus_0,quality_minus_1,quality_minus_2,quality_minus_5,quality_minus_6,quality_minus_11,err_week_0,err_week_1,err_week_2,err_week_3,err_week_4,err_week_5,err_week_6,qual_week_0,qual_week_1,qual_week_2,qual_week_3,qual_week_4,qual_week_5,qual_week_6,err_hours_0,err_hours_1,err_hours_2,err_hours_3,err_hours_4,err_hours_5,err_hours_6,err_hours_7,err_hours_8,err_hours_9,err_hours_10,err_hours_11,err_hours_12,err_hours_13,err_hours_14,err_hours_15,err_hours_16,err_hours_17,err_hours_18,err_hours_19,err_hours_20,err_hours_21,err_hours_22,err_hours_23,qual_hours_0,qual_hours_1,qual_hours_2,qual_hours_3,qual_hours_4,qual_hours_5,qual_hours_6,qual_hours_7,qual_hours_8,qual_hours_9,qual_hours_10,qual_hours_11,qual_hours_12,qual_hours_13,qual_hours_14,qual_hours_15,qual_hours_16,qual_hours_17,qual_hours_18,qual_hours_19,qual_hours_20,qual_hours_21,qual_hours_22,qual_hours_23
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,317.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,144.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.0,41.0,45.0,45.0,43.0,54.0,52.0,12.0,0.0,0.0,0.0,0.0,0.0,12.0,33.0,13.0,12.0,28.0,17.0,45.0,28.0,16.0,11.0,8.0,8.0,5.0,5.0,0.0,10.0,9.0,6.0,15.0,19.0,12.0,0.0,10.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0
1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0,0.0,0.0,2365.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,164.0,138.0,107.0,166.0,1534.0,117.0,139.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62.0,34.0,68.0,27.0,21.0,0.0,2.0,22.0,29.0,57.0,28.0,83.0,160.0,169.0,183.0,193.0,210.0,153.0,85.0,230.0,176.0,258.0,70.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,306.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,27.0,44.0,552.0,0.0,0.0,372.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,52.0,34.0,46.0,40.0,34.0,42.0,58.0,24.0,0.0,36.0,12.0,12.0,0.0,12.0,4.0,3.0,12.0,25.0,6.0,21.0,17.0,7.0,10.0,15.0,20.0,8.0,0.0,6.0,4.0,27.0,5.0,6.0,5.0,28.0,24.0,30.0,13.0,10.0,12.0,36.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0,0.0,0.0,306.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,51.0,52.0,42.0,45.0,37.0,22.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,15.0,8.0,18.0,37.0,8.0,9.0,45.0,0.0,4.0,12.0,1.0,3.0,0.0,0.0,8.0,3.0,5.0,5.0,19.0,39.0,30.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0,777.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,87.0,1044.0,0.0,0.0,36.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,96.0,144.0,115.0,59.0,171.0,84.0,108.0,12.0,0.0,12.0,0.0,0.0,0.0,0.0,7.0,0.0,5.0,133.0,0.0,12.0,9.0,77.0,30.0,30.0,10.0,8.0,11.0,36.0,8.0,7.0,18.0,22.0,

In [42]:
df['model_0']

0          0.0
1          0.0
2          0.0
3          0.0
4        777.0
         ...  
14995      0.0
14996      0.0
14997    826.0
14998    155.0
14999      0.0
Name: model_0, Length: 15000, dtype: float64

### 1.2 Data preprocess

In [43]:
# 변수 이름 변경
# error  -> train_x
# problem-> train_y

# train_x = df.to_numpy()
train_x = df.to_numpy()
train_y = problem.to_numpy()
# del error, problem
print(train_x.shape)
print(train_y.shape)

(15000, 129)
(15000,)


### 1.3 Model Train

In [44]:
# Train
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5

lr = 0.004 #learning_rate
nbr = 20000 #num_boost_round 
vb = 300 #verbose_eval
esr = 1000 #early_stopping_rounds
boosting_type = ['gbdt','dart']
# 파라미터 설정
params =      {
                'boosting_type' : boosting_type[0],
                'objective'     : 'binary',
                'metric'        : 'auc',
                'learning_rate' : lr,
                'seed': 2021
                }
es_only_index = 0
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=2021)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = nbr,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = vb, 
                        early_stopping_rounds = esr
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')
    es_only_index += 1


print(f"RESULT AUC : {np.mean(auc_scores)}")

[LightGBM] [Info] Number of positive: 3981, number of negative: 8019
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012511 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 16660
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 129
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.331750 -> initscore=-0.700281
[LightGBM] [Info] Start training from score -0.700281
Training until validation scores don't improve for 1000 rounds
[300]	valid_0's auc: 0.80487	valid_0's pr_auc: 0.7104
[600]	valid_0's auc: 0.812942	valid_0's pr_auc: 0.720179
[900]	valid_0's auc: 0.816405	valid_0's pr_auc: 0.731025
[1200]	valid_0's auc: 0.817055	valid_0's pr_auc: 0.731868
[1500]	valid_0's auc: 0.817216	valid_0's pr_auc: 0.732659
[1800]	valid_0's auc: 0.815994	valid_0's pr_auc: 0.732096
[2100]	valid_0's auc: 0.815758	val

## 2. MODEL Predict

### 2.1 TEST Data Load

In [47]:
test_df  = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DACON/LG_predict_complain/20210203_test_data_merge_2.csv')
test_df.drop(['user_id'],axis=1, inplace=True)
display(test_df.head())

,err0,err1,err2,err3,err4,err5,err6,err7,err8,err9,err10,err11,err12,err13,err14,err15,err16,err17,err18,err19,err20,err21,err22,err23,err24,err25,err26,err27,err28,err29,err30,err31,err32,err33,err34,err35,err36,err37,err38,err39,err40,err41,model_0,model_1,model_2,model_3,model_4,model_5,model_6,model_7,model_8,03.11.1141,03.11.1149,03.11.1167,04.16.2641,04.16.3345,04.16.3439,04.16.3553,04.16.3569,04.16.3571,04.22.1170,04.22.1442,04.22.1448,04.22.1478,04.22.1608,04.22.1656,04.22.1666,04.22.1684,04.22.1750,04.22.1772,04.22.1778,04.33.1095,04.33.1125,04.33.1149,04.33.1171,04.33.1185,04.33.1261,04.73.2237,04.73.2569,04.73.2571,04.73.2577,04.82.1684,04.82.1730,04.82.1778,05.15.2090,05.15.2092,05.15.2114,05.15.2120,05.15.2122,05.15.2138,05.15.3104,05.66.3237,05.66.3571,10,10.22.1770,10.22.1780,8.5.3,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,quality_minus_0,quality_minus_1,quality_minus_2,quality_minus_3,quality_minus_4,quality_minus_5,quality_minus_6,quality_minus_7,quality_minus_8,quality_minus_9,quality_minus_10,quality_minus_11,quality_minus_12,err_week_0,err_week_1,err_week_2,err_week_3,err_week_4,err_week_5,err_week_6,qual_week_0,qual_week_1,qual_week_2,qual_week_3,qual_week_4,qual_week_5,qual_week_6,err_hours_0,err_hours_1,err_hours_2,err_hours_3,err_hours_4,err_hours_5,err_hours_6,err_hours_7,err_hours_8,err_hours_9,err_hours_10,err_hours_11,err_hours_12,err_hours_13,err_hours_14,err_hours_15,err_hours_16,err_hours_17,err_hours_18,err_hours_19,err_hours_20,err_hours_21,err_hours_22,err_hours_23,qual_hours_0,qual_hours_1,qual_hours_2,qual_hours_3,qual_hours_4,qual_hours_5,qual_hours_6,qual_hours_7,qual_hours_8,qual_hours_9,qual_hours_10,qual_hours_11,qual_hours_12,qual_hours_13,qual_hours_14,qual_hours_15,qual_hours_16,qual_hours_17,qual_hours_18,qual_hours_19,qual_hours_20,qual_hours_21,qual_hours_22,qual_hours_23
0,0.0,0.0,0.0,0.0,62.0,1.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,3.0,67.0,37.0,9.0,3.0,0.0,1.0,0.0,983.0,1308.0,5.0,2.0,32.0,0.0,0.0,0.0,0.0,83.0,2.0,16.0,0.0,1.0,3.0,3.0,0.0,2.0,92.0,0.0,2.0,0.0,2687.0,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,1.0,0.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,60.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,267.0,62.0,410.0,254.0,223.0,1024.0,510.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,37.0,36.0,75.0,132.0,24.0,44.0,43.0,36.0,12.0,123.0,30.0,11.0,32.0,31.0,105.0,284.0,157.0,133.0,143.0,522.0,475.0,97.0,113.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0
1,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,12.0,16.0,4.0,6.0,53.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,284.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2560.0,0.0,0.0,0.0,0.0,30756.0,0.0,0.0,9.0,9.0,9.0,0.0,0.0,9.0,9.0,0.0,0.0,0.0,0.0,9.0,0.0,43.0,32.0,57.0,24.0,18.0,54.0,56.0,12.0,0.0,12.0,12.0,0.0,0.0,0.0,10.0,6.0,8.0,2.0,20.0,19.0,21.0,4.0,18.0,5.0,20.0,12.0,25.0,1.0,9.0,5.0,4.0,14.0,9.0,24.0,12.0,8.0,9.0,19.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,79.0,2.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,1.0,173.0,133.0,0.0,0.0,0.0,0.0,0.0,60.0,40.0,2.0,0.0,40.0,0.0,0.0,0.0,0.0,230.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,14.0,113.0,1.0,2.0,941.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,133.0,0.0,0.0,0.0,0.0,1596.0,0.0,0.0,15.0,15.0,15.0,0.0,0.0,15.0,15.0,0.0,

### 2.2 DATA preprocess

In [48]:
test_df.drop(all_data_col_fwver,axis=1,inplace=True)
# df.drop(all_data_col_errcode,axis=1,inplace=True)
# test_df.drop(all_data_col_qual_weekday,axis=1,inplace=True)
# test_df.drop(all_data_col_qual_hours,axis=1,inplace=True)
drop_list = ['err28', 'quality_3', 'quality_4', 'quality_minus_3', 'quality_minus_4', 'quality_minus_7', 'quality_minus_8', 'quality_minus_9', 'quality_minus_10', 'quality_minus_12']
test_df.drop(drop_list, axis=1,inplace=True)
test_df.shape

(14999, 129)

### 2.3 TEST data predict

In [49]:
# 예측
pred_y_list = []
for model in models:
    # pred_y = model.predict(test_x)
    pred_y = model.predict(test_df.to_numpy())
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

In [50]:
pred_ensemble

array([[0.93116459],
       [0.23050819],
       [0.28930899],
       ...,
       [0.49343188],
       [0.90310151],
       [0.34464687]])

In [51]:
today = '20210130'
sample_submssion = pd.read_csv(PATH+'sample_submission.csv')
sample_submssion['problem'] = pred_ensemble.reshape(-1)
sample_submssion.to_csv(f"dacon_baseline_t{threshold}_lr{lr}_nbr{nbr}_stop{esr}_{today}_col130_auc{np.mean(auc_scores)}.csv", index = False)
# sample_submssion.to_csv(f"dacon_baseline_t{threshold}_lr{lr}_nbr{nbr}_stop{esr}_{today}_auc{np.mean(auc_scores)}_gbdt_hpt1.csv", index = False)
sample_submssion

,user_id,problem
0,30000,0.931165
1,30001,0.230508
2,30002,0.289309
3,30003,0.721313
4,30004,0.735966
...,...,...
14994,44994,0.322146
14995,44995,0.321896
14996,44996,0.493432
14997,44997,0.903102
